# Basic ML Models of the Kaggle House Price Dataset

In [1]:
import pandas as pd
print("pandas version: {}". format(pd.__version__))

# numpy: support for large, multi-dimensional arrays and matrices and high-level mathematical functions
import numpy as np
print("numpy version: {}". format(np.__version__))

import sklearn
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso, Ridge, BayesianRidge, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import ShuffleSplit, train_test_split, cross_val_score, learning_curve
from sklearn.feature_selection import VarianceThreshold
from sklearn.compose import TransformedTargetRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
print("sklearn version: {}". format(sklearn.__version__))

import optuna
print("optuna version: {}". format(optuna.__version__))

import mlflow
from mlflow.utils.mlflow_tags import MLFLOW_PARENT_RUN_ID
from mlflow.tracking import MlflowClient
print("mlflow version: {}". format(mlflow.__version__))

import matplotlib.pyplot as plt
import seaborn as sns

import os
from datetime import datetime
from typing import Optional
import warnings
warnings.simplefilter('ignore')

import yaml
with open('ml_parameter.yaml') as file:
  config_data= yaml.safe_load(file)


from create_algorithm import create_algorithm
import pipeline_classes

pandas version: 1.5.3
numpy version: 1.23.5
sklearn version: 1.2.2


e:\programs\anaconda3\envs\mlflow_optuna\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


optuna version: 2.10.1
mlflow version: 2.1.1


In [2]:
VERSION = "1"
SCRIPT = "test"

# VERSION = 0.6
# SCRIPT = "houseprices_BasicModels_pipeline"

mlflow.set_tracking_uri("http://localhost:5000")
client = MlflowClient()
try:
    experiment = client.create_experiment(f"{SCRIPT}_{VERSION}")
except:
    experiment = client.get_experiment_by_name(f"{SCRIPT}_{VERSION}").experiment_id

In [3]:
def StratifiedKFold_Regression(df, target, bins, n_splits):
    from sklearn.model_selection import StratifiedKFold

    df['Fold'] = -1
    skf = StratifiedKFold(n_splits=n_splits)

    y_cat = pd.cut(df[target], bins, labels=False)

    for fold_no, (t, v) in enumerate(skf.split(y_cat, y_cat)):
        df.loc[v, 'Fold'] = fold_no

    sns.histplot(data=df, x=y_cat, hue="Fold", multiple="stack")

    return df

In [4]:
def load_data():
    # load raw training and test dataset
    df_train = pd.read_csv("../01_RawData/train.csv")
    df_test = pd.read_csv("../01_RawData/test.csv")

    # drop features with more then 30% of missing values
    df_train = df_train.drop(["Id", "PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu"], axis=1)
    df_test = df_test.drop(["Id", "PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu"], axis=1)

    # compute StratifiedKFold column
    df_train = StratifiedKFold_Regression(df_train, "SalePrice", 50, 5)

    # split the training and test dataset to the input features (x_train, x_test) and the survival class (y_train)
    y_train = df_train['SalePrice']
    x_train = df_train.drop(['SalePrice'], axis=1)
    x_test = df_test

    x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

    return x_train, y_train, x_validate, y_validate, x_test

x_train, y_train, x_validate, y_validate, x_test = load_data()

In [5]:
# define cross validation

# cv = ShuffleSplit(
#     n_splits = 10,
#     test_size = 0.2,
#     random_state = 42
#     )

from sklearn.model_selection import GroupKFold
cv = GroupKFold(n_splits=5)

In [6]:
def create_model(self, trial):

    child_run = client.create_run(
        experiment_id=experiment,
        tags={
            MLFLOW_PARENT_RUN_ID: self.parent_run.info.run_id
        }
    )

    model = create_algorithm(self.model_type, trial, client, child_run, config_data)

    # create parameter for data preprocessing pipeline
    imputer_function_numeric=trial.suggest_categorical("preprocessing_imputer_function_numeric", ["SimpleImputer", "KNNImputer"])
    transform_skewed_features_flag=trial.suggest_categorical("preprocessing_transform_skewed_features_flag", [True, False])
    ohe_min_frequency=trial.suggest_float("preprocessing_ohe_min_frequency", 0, 0.2, log=False)
    ohe_max_categories=trial.suggest_int('preprocessing_ohe_max_categories', 5, 20)
    feature_selection_low_variance_flag=trial.suggest_float("preprocessing_feature_selection_low_variance_flag", 0.7, 0.99, log=False)
    correlation=trial.suggest_float("preprocessing_correlation", 0.7, 0.99, log=False)
    scaler = trial.suggest_categorical("columnprep__transformers_num", ["StandardScaler", "MinMaxScaler"])
    pca_n_components = trial.suggest_float('pca_n_components', 0, 1)

    # log all parameters of the data preprocessing with mlflow
    client.log_param(child_run.info.run_id, "preprocessing_imputer_function_numeric", imputer_function_numeric)
    client.log_param(child_run.info.run_id, "preprocessing_transform_skewed_features_flag", transform_skewed_features_flag)
    client.log_param(child_run.info.run_id, "preprocessing_ohe_min_frequency", ohe_min_frequency)
    client.log_param(child_run.info.run_id, "preprocessing_ohe_max_categories", ohe_max_categories)
    client.log_param(child_run.info.run_id, "preprocessing_feature_selection_low_variance_flag", feature_selection_low_variance_flag)
    client.log_param(child_run.info.run_id, "preprocessing_correlation", correlation)
    client.log_param(child_run.info.run_id, "preprocessing_scaler", scaler)  
    client.log_param(child_run.info.run_id, "pca_n_components", pca_n_components)


    pipeline = make_pipeline(
            pipeline_classes.ImputMissingValuesNumeric(imputer_function=imputer_function_numeric),
            pipeline_classes.ImputMissingValuesCategoric(),
            pipeline_classes.CreateNewFeatures(),
            pipeline_classes.ImputMissingValuesNumeric(imputer_function=imputer_function_numeric),
            pipeline_classes.SkewedFeatureTransformer(transform_skewed_features_flag=transform_skewed_features_flag),
            pipeline_classes.LabelEncoderTransformer(),
            pipeline_classes.OneHotEncoderTransformer(ohe_min_frequency=ohe_min_frequency, ohe_max_categories=ohe_max_categories),
            pipeline_classes.LowVarianceTransformer(variance_threshold=feature_selection_low_variance_flag),
            pipeline_classes.CorrelationTransformer(correlation_threshold=correlation),
            pipeline_classes.ScalerTransformer(columnprep__transformers_num=scaler),
            PCA(n_components=pca_n_components),
            model
    )

    return pipeline, child_run

In [7]:
def create_submission(best_model, x_test, parent_run, study):
    # check if folder exists
    if not os.path.exists(f'submissions/{SCRIPT}/{VERSION}'):
        os.makedirs(f'submissions/{SCRIPT}/{VERSION}')
    
    # predict the test values with the training classification model
    y_pred = best_model.predict(x_test)
    
    df_submission = pd.read_csv("../01_RawData/sample_submission.csv")
    df_submission.iloc[:, 1] = y_pred
    
    df_submission.to_csv(f'submissions/{SCRIPT}/{VERSION}/{parent_run.info.run_id}.csv', index=False)

In [8]:
def evaluate_model(x_train, y_train, y_validate, y_validate_pred, pipeline, parent_run):

    def plot_learning_curve(pipeline, x_train, y_train, cv=cv, train_sizes=np.linspace(.1, 1.0, 50)):
        train_sizes, train_scores, test_scores = learning_curve(
            pipeline,
            x_train,
            y_train,
            cv=cv,
            n_jobs=-1,
            groups=x_train["Fold"],
            train_sizes=np.linspace(.1, 1.0, 8)
            )


        fig1, ax1 = plt.subplots()
        ax1.set_xlabel("Training examples")
        ax1.set_ylabel("Score")
        train_scores_mean = np.mean(train_scores, axis=1)
        train_scores_std = np.std(train_scores, axis=1)
        test_scores_mean = np.mean(test_scores, axis=1)
        test_scores_std = np.std(test_scores, axis=1)
        ax1.grid()

        ax1.fill_between(train_sizes, train_scores_mean - train_scores_std,
                        train_scores_mean + train_scores_std, alpha=0.1,
                        color="r")
        ax1.fill_between(train_sizes, test_scores_mean - test_scores_std,
                        test_scores_mean + test_scores_std, alpha=0.1, color="g")
        ax1.plot(train_sizes, train_scores_mean, 'o-', color="r",
                label="Training score")
        ax1.plot(train_sizes, test_scores_mean, 'o-', color="g",
                label="Cross-validation score")

        ax1.legend(loc="best")
        ax1.set_title("Difference between training and CV: "\
            + str(round(test_scores_mean[7] / train_scores_mean[7] * 100, 2))\
            + "%")
        client.log_figure(parent_run.info.run_id, fig1, 'plot_learning_curve.png')
        plt.close()

    plot_learning_curve(pipeline, x_train, y_train)


    def plot_scatter(y_true, y_pred):
        fig2, ax2 = plt.subplots()
        ax2.scatter(y_pred, y_true.values)
        ax2.plot([min(y_pred), max(y_pred)], [min(y_pred), max(y_pred)], c="red")
        ax2.set_ylabel("True Values")
        ax2.set_xlabel("Predicted Values")
        client.log_figure(parent_run.info.run_id, fig2, 'plot_regression.png')
        plt.close()

    plot_scatter(y_validate, y_validate_pred)

In [9]:
class Objective:
    
    def __init__(self, model_type, parent_run):
        self.best_model = None
        self._model = None
        self.model_type = model_type
        self.parent_run = parent_run

    
    def __call__(self, trial):
    
        pipeline, child_run = create_model(self, trial)
        self._model = pipeline

        cvs = np.sqrt(
            -cross_val_score(
                pipeline,
                x_train.drop(columns=["Fold"]), y_train,
                cv=cv,
                groups=x_train["Fold"],
                scoring="neg_mean_squared_error",
                n_jobs=1
                )
            )

        rmse = cvs.mean()
        rmse_var = np.var(cvs)

        client.log_metric(child_run.info.run_id, "cv_rmse", rmse)
        client.log_metric(child_run.info.run_id, "cv_var", rmse_var)
        
        return rmse

    def callback(self, study, trial):
        if study.best_trial == trial:
            self.best_model = self._model

In [10]:
model_type='Lasso'
# model_type='Ridge'
# model_type='BayesianRidge'
# model_type='ElasticNet'
# model_type='GradientBoostingRegressor'
# model_type='RandomForestRegressor'





parent_run = client.create_run(
    experiment_id=experiment, 
    tags={"script_name": SCRIPT, "script_version": VERSION}
    )

objective = Objective(model_type, parent_run)

study = optuna.create_study(direction="minimize")

study.optimize(
    objective,
    n_trials=config_data["N_TRAILS"],
    timeout=config_data["TIMEOUT"],
    n_jobs=-1,
    callbacks=[objective.callback]
    )

print("Best trial:")
print(study.best_value)
print(study.best_params)

print("Log CV Params")
client.log_metric(parent_run.info.run_id, "best_cv_score", round(study.best_value, 3))
# client.log_param(parent_run.info.run_id, "transformer_num", str(transformer_num))

# client.log_param(parent_run.info.run_id, "cv_n_splits", cv.n_splits)
# client.log_param(parent_run.info.run_id, "cv_train_size", cv.train_size)
# client.log_param(parent_run.info.run_id, "cv_test_size", cv.test_size)
# client.log_param(parent_run.info.run_id, "cv_random_state", cv.random_state)

for param in study.best_params:
    client.log_param(parent_run.info.run_id, param, study.best_params[param])

best_model = objective.best_model
client.log_param(parent_run.info.run_id, "algo", best_model.get_params()["steps"][-1][1].__class__.__name__)


# fit the pipeline to compute the validation results
print("Fit Best Model")
best_model.fit(x_train.drop(columns=["Fold"]), y_train)

# predict the training outcome
print("Predict Best Model")
y_validate_pred = best_model.predict(x_validate)

# evaluate model
print("Evaluate Best Model")
client.log_metric(parent_run.info.run_id, "r2", r2_score(y_validate, y_validate_pred))
evaluate_model(x_train, y_train, y_validate, y_validate_pred, best_model, parent_run)

# # create submission of best model
print("Create submission")
create_submission(best_model, x_test, parent_run, study)

mlflow.end_run()

[I 2023-04-12 20:24:40,849] A new study created in memory with name: no-name-97ff1f5b-27e3-413a-8439-a3ecb076990b
[I 2023-04-12 20:24:44,007] Trial 3 finished with value: 46278.8924560231 and parameters: {'lasso_alpha': 4.323549774562297, 'lasso_max_iter': 4837, 'preprocessing_imputer_function_numeric': 'KNNImputer', 'preprocessing_transform_skewed_features_flag': True, 'preprocessing_ohe_min_frequency': 0.008614243203605843, 'preprocessing_ohe_max_categories': 18, 'preprocessing_feature_selection_low_variance_flag': 0.8718566219908117, 'preprocessing_correlation': 0.8861844903007997, 'columnprep__transformers_num': 'MinMaxScaler', 'pca_n_components': 0.588324692796322}. Best is trial 3 with value: 46278.8924560231.
[I 2023-04-12 20:24:53,849] Trial 1 finished with value: 35606.55360523241 and parameters: {'lasso_alpha': 4.818407168768707, 'lasso_max_iter': 6823, 'preprocessing_imputer_function_numeric': 'SimpleImputer', 'preprocessing_transform_skewed_features_flag': True, 'preprocess

Best trial:
35442.70067331441
{'lasso_alpha': 3.4535701765357087, 'lasso_max_iter': 7968, 'preprocessing_imputer_function_numeric': 'SimpleImputer', 'preprocessing_transform_skewed_features_flag': False, 'preprocessing_ohe_min_frequency': 0.07217638543594829, 'preprocessing_ohe_max_categories': 14, 'preprocessing_feature_selection_low_variance_flag': 0.8734809297874558, 'preprocessing_correlation': 0.7640610270802362, 'columnprep__transformers_num': 'MinMaxScaler', 'pca_n_components': 0.9726148253847371}
Log CV Params
Fit Best Model
Predict Best Model
Evaluate Best Model
Create submission


In [11]:
mlflow.end_run()